# hw4 과제 (4점, 제출기한: 11/15 밤 10시까지)
* 과목: 컴파일러
* 이름:
* 학번

 1. `string :: Parser String`을 작성하라 (1점)
 1. `many1`와 `many`의 결과가 가장 길게 매칭되는 것만 나오게 바꾸어 보라. (1점)
 1. `float :: Parser Float`을 작성하라 (2점)

In [1]:
type Parser a = String -> [(a,String)]

In [2]:
anyChar :: Parser Char
anyChar (c:cs) = [(c, cs)]
anyChar _      = []

In [3]:
anyChar "abcde"
anyChar "hijkl"

[('a',"bcde")]

[('h',"ijkl")]

In [4]:
char :: Char -> Parser Char
char c' (c:cs) | c'==c = [(c,cs)]
char _  _              = []

In [5]:
char 'a' "abcde"
char 'a' "hijkl"

[('a',"bcde")]

[]

In [6]:
oneOf xs (c:cs) | c `elem` xs = [(c,cs)]
oneOf _  _                    = []

In [7]:
oneOf ['a','b','c'] "abcde"
oneOf ['a','b','c'] "bcdef"
oneOf ['a','b','c'] "cdefg"
oneOf ['a','b','c'] "defgh"

[('a',"bcde")]

[('b',"cdef")]

[('c',"defg")]

[]

In [8]:
satisfy :: (Char -> Bool) -> Parser Char
satisfy p (c:cs) | p c = [(c,cs)] 
satisfy _  _           = []

In [9]:
anyChar = satisfy (const True) -- statisfy (\_ -> True)
char x = satisfy (x==)
oneOf xs = satisfy (`elem` xs)

In [10]:
anyChar "abcde"
anyChar "hijkl"

[('a',"bcde")]

[('h',"ijkl")]

In [11]:
char 'a' "abcde"
char 'a' "hijkl"

[('a',"bcde")]

[]

In [12]:
oneOf ['a','b','c'] "abcde"
oneOf ['a','b','c'] "bcdef"
oneOf ['a','b','c'] "cdefg"
oneOf ['a','b','c'] "defgh"

[('a',"bcde")]

[('b',"cdef")]

[('c',"defg")]

[]

In [13]:
import Data.Char
space = satisfy isSpace
alphaNum = satisfy isAlphaNum
letter = satisfy isAlpha
lower = satisfy isLower
upper = satisfy isUpper
digit = satisfy isDigit

In [14]:
digit "abcde"
digit "123abc"

[]

[('1',"23abc")]

아래의 실행예처럼 주어진 문자열과 정확히 일치하는 만큼을 분석하여 처리하는 `string` 함수를 작성하라. (다음 과제의 첫 문제가 될 내용이다)
```haskell
string "abc" "abcdef" == [("abc","def")]
```

In [15]:
string :: String -> Parser String 
string ""     cs                 = [("",cs)]
string (x:xs) (c:cs) | undefined = undefined
string _      _                  = []

----

다음 과제의 두번째 문제는 아래 모든 개수를 다 고려하는 문법분석 함수 many1과 many를 가장 길게 매칭되는 경우만 결과로 하는 것으로 수정하라는 것

In [16]:
-- parse one or more times
many1 :: Parser a -> Parser [a]
many1 p cs = [([x], cs') | (x,cs')<-ps] -- 한번 처리한 결과 x를 리스트로
            ++
             [(x:xs, cs'') | (x,cs')<-ps, (xs,cs'')<-many1 p cs']
           where ps = p cs

-- parse zero or more time
many :: Parser a -> Parser [a]
many p cs = [([],cs)] ++ many1 p cs

In [17]:
letter "hello world"

[('h',"ello world")]

In [18]:
many1 letter "hello world"
many1 letter "   hello world"

[("h","ello world"),("he","llo world"),("hel","lo world"),("hell","o world"),("hello"," world")]

[]

In [19]:
many letter "hello world"
many letter "   hello world"

[("","hello world"),("h","ello world"),("he","llo world"),("hel","lo world"),("hell","o world"),("hello"," world")]

[("","   hello world")]

In [20]:
spaces = many space

In [21]:
spaces "hello world"
spaces "   hello world"

[("","hello world")]

[("","   hello world"),(" ","  hello world"),("  "," hello world"),("   ","hello world")]

## 십진수 양의 정수를 분석하기

In [22]:
-- 맨 앞에는 0이 아닌 숫자, 그 다음에는 아무 십진수 숫자나

firstDigit = oneOf ['1'..'9'] -- 맨 앞글자를 처리 (딱 1개 글자)
digits = many digit       -- 뒷글자들을 처리 (0개 이상 글자)

In [23]:
firstDigit "123 abc"
firstDigit "023 abc"

[('1',"23 abc")]

[]

In [24]:
digits "123 abc"
digits "023 abc"

[("","123 abc"),("1","23 abc"),("12","3 abc"),("123"," abc")]

[("","023 abc"),("0","23 abc"),("02","3 abc"),("023"," abc")]

In [25]:
:type firstDigit
:type digits

firstDigit :: Parser Char

digits :: Parser [Char]

이제 `firstDigit`으로 분석한 결과값(`x`)과 처리하고 난 나머지 문자열을 `digits`으로 분석한 결과값(`xs`)을 합친 값(`x:xs`)을 그 결과값으로 하는 Parser를 정의해 보자.

In [26]:
posint:: Parser String
posint s = [(x:xs, cs')  | (x, cs)<-firstDigit s, (xs, cs') <- digits cs]

In [27]:
:type read

read :: forall a. Read a => String -> a

In [28]:
read "123" :: Int

123

In [29]:
posint :: Parser Int
posint cs = [(read(x:xs), cs'')  | (x, cs')<-firstDigit cs, (xs, cs'') <- digits cs']

In [30]:
posint "123 hello"

[(1,"23 hello"),(12,"3 hello"),(123," hello")]

하나의 Parser 다음에 다른 Parser를 순차적으로 실행하는 패턴은 자주 나타날 수 있기 때문에 이것을 다음과 같은 연산자로 일반화할 수 있다.

In [31]:
(>>=) :: Parser a -> (a -> Parser b) -> Parser b
p >>= f = \cs -> [(y,cs'') | (x, cs')<-p cs, (y,cs'') <- f x cs']

infixr 1 >>=

일반적으로 이런 연산자를 여러 번 겹쳐 다음과 같이 쓰면 여러 Parser들을 차례로 실행한 새로운 Parser를 정의할 수 있다.  
```haskell
p1 >>= \x1 -> p2 >>= \x2 -> p3 >>= \x3 -> ....
```

괄호를 많이 써주자면 아래와 같다
```haskell
p1 >>= (\x1 -> (p2 >>= (\x2 -> (p3 >>= (\x3 -> ....))))
```

In [32]:
posint :: Parser Int
posint = firstDigit >>= \x  ->
         digits     >>= \xs ->
         \cs -> [(read(x:xs), cs)]

In [33]:
posint "123 hello"

[(1,"23 hello"),(12,"3 hello"),(123," hello")]

위에서 마지막 줄인 이 부분은
```haskell
\cs -> [(read(x:xs), cs)]
```
두 Parser를 순차적으로 실행한 결과를 모아서 만든 어떤 새로운 결과값 하나만 돌려주고
그 두 Parser를 실행하고 남은 나머지 문자열인 `cs`에 대해서는 어떠한 추가적인 문법 처리도 일어나지 않는다.

이런 식으로 순차적으로 이어붙이는 연산의 마지막에서는 위와 같은 패턴을 자주 사용할 것이라는 것이 예상되므로
이를 아래와 같이 이를 `return` 함수로 정의하자. 

In [34]:
return :: b -> Parser b
return v = \cs -> [(v,cs)] 

In [35]:
posint :: Parser Int
posint = firstDigit >>= \x  ->
         digits     >>= \xs ->
         return (read(x:xs))

# 자연수 Parser와  정수 Parser

이제 자연수 Parser와 정수 Parser를 만들어 보자. 자연수는 양의 정수이거나 0이며 정수는
자연수이거나 음의 정수이므로 이들 각각에 해당하는 Parser를 만들어 결과가 그 중 어느 것이든 될 수 있는 Parser를 만들면 된다. 이를 위해 필요한 Parser에 대한 연산자를 정의하자.

In [36]:
(<|>) :: Parser a -> Parser a -> Parser a
p1 <|> p2 = \cs -> p1 cs ++ p2 cs

infixl 3 <|>

In [37]:
nat :: Parser Int
nat = zero <|> posint

int :: Parser Int
int = nat <|> negint

zero :: Parser Int
zero   = char '0' >>= \_ ->
         return 0

negint :: Parser Int
negint = char '-' >>= \_ ->
         posint   >>= \n ->
         return (-n)

In [38]:
nat "123 hello"
nat "0 hello"
nat "-12 hello"

[(1,"23 hello"),(12,"3 hello"),(123," hello")]

[(0," hello")]

[]

In [39]:
int "123 hello"
int "0 hello"
int "-12 hello"

[(1,"23 hello"),(12,"3 hello"),(123," hello")]

[(0," hello")]

[(-1,"2 hello"),(-12," hello")]

`-0.025`, `0.025`, `3.1415927`, `-31.45`, `3.0`, `3`, `-4.0`, `-4`, `0.0`, `0` 등과 같은 형태를 분석하는
`float :: Parser Float`을 작성하라.

In [40]:
float :: Parser Float
float = undefined

# 심화자율학습 주제

부교재 12장 내용으로 소개되는 내용인 하스켈에서 Monad와 Applicative에 대해 알아보라.